# Building an RNN for Transcription Factor Binding Site Prediction

***

This Jupyter Notebook Implements the KEGRU model, a recurrent neural network for predicting [transcription factor](https://en.wikipedia.org/wiki/Transcription_factor) (TF) binding sites. The KEGRU model was first proposed in: "Recurrent Neural Network for Predicting Transcription Factor Binding Sites".<br />
The original 2018 paper by Shen Z, Bao W \& Huang D-S is freely available and can be found here: https://www.nature.com/articles/s41598-018-33321-1

This paper made some important contributions to deep learning in the field of Bioinformatics. First, they introduced the idea of "k-mer embeddings" for DNA/RNA sequence via the popular [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) pre-trained neural network. Second, they implemented a gold-standard Recurrent Neural Network (RNN), namely, a Bidirectional Gated Recurrent Unit (BiGRU) Neural Network to operate on this k-mer embedding that improves upon previous deep-learning results. They display the success of these models in working on small vocabulary, pattern-based textual data such as DNA. 

***

In this tutorial we will walk through a step-by-step guide to building their BiGRU Neural Network that performs classification of a k-mer embedded dataset. The model classifies an instance as either a TF binding site (**1**) or not a TF binding site (__0__).


The following packages are required to run this notebook:

In [1]:
# import statements for support packages 
import csv
import gzip
import math
import random
import numpy as np
from sklearn import metrics

# import statements for building the BiGRU
import torch
import gensim 
import torchvision
import torch.nn as nn
import torch.nn.functional as func 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset 

## Preparing Chip-Seq Datasets

***
Next we use load_chipseq to parse the *.seq.gz file into one-third training, validation and test sets. The shuffle_seq function is used to create instances of negative cases. This is required when using peak-called files such as *.seq files. 

In [2]:
def shuffle_seq(seq, k):
    '''returns a shuffled version of the original sequence.'''
    if k > len(seq) or k < 1:
        k = len(seq)
        
    b = [seq[i:i+k] for i in range(0, len(seq), k)]
    random.shuffle(b)
    d = ''.join([str(x) for x in b])
    return d


def load_chipseq(file_name):
    '''parses gzip compressed *.seq files. Returns
    one-third split data and the raw sequences.'''
    all_data = []
    seq_data = []
    
    with gzip.open(file_name, 'rt') as peak:
        next(peak)
        reader = csv.reader(peak, delimiter = '\t')

        for row in reader:
            all_data.append([row[2],[1]])
            all_data.append([shuffle_seq(row[2], 2),[0]])
            seq_data.append(row[2])

    split = int(len(all_data)/3)
    training = all_data[:split]
    validation = all_data[split:split+split]
    test = all_data[split+split:]
        
    return training, validation, test , all_data, seq_data

In [4]:
# make train, validation, and test sets
train, valid, test, full_data, seq_data = load_chipseq('ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz')
print(train[0], "\n", train[1])

['CCCGCTCCCTAGCAACGATTGGTTAACGAGGCCTGGTTTCCAGAAATGGGCACTATTTCCGGACGATGTTTGTAGAAGGGAGATCGCTGGGCGGGCGGACT', [1]] 
 ['CTTGAATGGAGAGGCTTCAACCTAGCTGCGGAGGCAATGCCTCACTCGTTGGTACAGAGCCATTTGGAGTTGGGATTCACCGGCTAGACCGGGGGACGCTT', [0]]


## Learn the k-mer Embedding Vectors

***
Here we will now take the sequence data generated above (*seq_data*) to create kmer corpus with *gen_kmer_bag*. We use a stride and kmer length as recommended in the paper. <br> We will then take this kmer corpus and learn vector embeddings with Word2Vec. Since the model complexity increases as the embedding dimension, embed_size, increases the minimum value tested in the paper was chosen: thus embed_size = 50.

Of the three weight initialistion methods chosen in the paper, "init-train",  is used as it showed consistently the best results over the four cell lines examined. Thus, we will use the vector embeddings, *embed.wv.vectors*, as the initialisation weights for the BiGRU (and we will later allow for them to be updated in training).  

In [5]:
def gen_kmer_bag(seq_list, kmer_len, s):
    '''returns a list of lists of kmers 
    according to a stride and kmer length.'''
    kmer_list = []
    for seq in seq_list:
        kmers = []
        for j in range(0,(len(seq) - kmer_len)+1, s):
              kmers.append(seq[j:j + kmer_len])  
        kmer_list.append(kmers)
        
    return kmer_list


# preset k-mer length and stride parameters according to the paper. 
kmer_len = 5
stride = 2
embed_size = 50 # chosen to minimise overfitting
embed_epochs = 100

# generate kmer corpus and train their embeddings on Word2Vec
bag_of_kmers = gen_kmer_bag(seq_data, kmer_len, stride)
embed = gensim.models.Word2Vec(bag_of_kmers, size = embed_size)
embed.train(bag_of_kmers, total_examples = len(bag_of_kmers), epochs = embed_epochs)

# first dimension of the initialisation weights to BiGRU
print(embed.wv.vectors[0]) 

[ 2.9376004  -2.247391   -8.597012   -5.345646   -5.8195224  -7.4170666
 10.487824    7.3334208  -0.6384367   6.461266   -1.4845182   2.7279136
  0.6967435  -3.1661236  -2.9139838   9.206927    6.919305   -0.71492606
  4.69404    -5.7016516   8.447689    5.550959    0.74414897 -2.9434175
 -1.667043   -0.08421978  3.717625    0.17041826 -0.9019605   4.1195397
 -3.7185333   3.390937    4.52332     8.690586   -5.767567   -1.4819341
 -0.7175306   3.925163    0.7670136   1.6557689   7.851567   -7.678966
 10.772115    3.0975785  -2.271372   -0.47215483  8.820241    0.20448932
 -1.8782592   5.412842  ]


## Initialise the Embedding Layer

***
Now we will define a class *kmer_embeddings* that will prepare the kmer embeddings for training the BiGRU. To achieve this *kmer_embeddings* must be a map-style dataset so that the instances of this class can be retrieved by DataLoader.

We use a mini_batch_size = 200 as used in the paper. We also create a dataset based upon the entire training and validation datasets that will be used to train the final model for preiction on the unseen test set. 

In [137]:
class kmer_embeddings(Dataset):

    def __init__(self, data_label = None, embed = None, kmer_len = 5, stride = 2):
        '''initialises the kmer_embeddings class variables. This class creates
        map-style datasets that can be used by the torch DataLoader.'''
        data = [x[0] for x in data_label]
        self.kmer_len = kmer_len
        self.stride = stride
        
        kmer_bag = gen_kmer_bag(data, self.kmer_len, self.stride)
        self.idx_data = torch.LongTensor(np.asarray([self.convert_data_to_index(kmers, embed.wv) for kmers in kmer_bag]))
        self.y_data = torch.from_numpy(np.asarray([y[1] for y in data_label], dtype = np.float32))
        self.len = len(self.idx_data)
      
    def __getitem__(self, idx):
        '''function required by DataLoader.'''
        return self.idx_data[idx], self.y_data[idx]

    def __len__(self):
        '''function required by DataLoader.'''
        return self.len

    def convert_data_to_index(self, kmers, wv):
        '''return index associated with each kmer if
        it exists in the word vector embeddings.'''
        idx_data = []
        for kmer in kmers:
            if kmer in wv:
                idx_data.append(wv.vocab[kmer].index)
        return idx_data

In [138]:
# present minibatch size according to the paper. 
mini_batch_size = 200

# create objects to be handled by DataLoader
train_dataset = kmer_embeddings(train, embed, kmer_len, stride)
valid_dataset = kmer_embeddings(valid, embed, kmer_len, stride)
test_dataset = kmer_embeddings(test, embed, kmer_len, stride)
train_val_split = int(2*len(full_data)/3)
final_dataset = kmer_embeddings(full_data[:train_val_split], embed, kmer_len, stride)

# prepare the datasets for the BiGRU
train_loader = DataLoader(dataset = train_dataset, batch_size = mini_batch_size, shuffle = True)
valid_loader = DataLoader(dataset = valid_dataset, batch_size = mini_batch_size, shuffle = False)
test_loader = DataLoader(dataset = test_dataset, batch_size = mini_batch_size, shuffle = False)
final_loader = DataLoader(dataset = final_dataset, batch_size = mini_batch_size, shuffle = True)

for i, (data, label) in enumerate(train_loader):
    print(len(data), len(data[0]))
    print(data)
    print(label.T)
    break

200 49
tensor([[883, 406,  34,  ..., 128,  51, 216],
        [364, 305,  20,  ..., 277, 262,  38],
        [ 45,  95, 234,  ...,  24,  38,   5],
        ...,
        [ 16, 570, 605,  ..., 448, 161, 224],
        [ 36,  17,  58,  ..., 107,  61, 171],
        [353, 616, 860,  ...,  29, 864, 732]])
tensor([[0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
         1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
         0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
         0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
         0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.

## Designing the BiGRU Model

***
Here we define the architecture of the BiGRU. The architecture is defined according to the BiGRU defined in the methods section of the paper. <br>
Since we are using torch as opposed to keras used in the paper we also have to specify a function for forward propagation. 

In [147]:
class biGRU(nn.Module):
    def __init__(self, hidden_size, drop_prob):
        '''initialses the architecture of the biGRU.'''
        super(biGRU, self).__init__()
        
        weights = torch.FloatTensor(embed.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights, freeze = False)
        self.hidden_size = hidden_size
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers = 1, bidirectional = True).to(device)
        self.hidden_weights = torch.normal(torch.zeros(2*hidden_size, 1)).to(device)
        
        self.hidden_bias = torch.randn(1).to(device)
        torch.nn.init.xavier_uniform_(self.hidden_weights)
        self.hidden_weights.requires_grad = True
        self.hidden_bias.requires_grad = True
        self.drop1 = nn.Dropout(p = drop_prob).to(device)
        self.FC2 = nn.Linear(20, 1, bias = True).to(device)
   

    def forward(self, x):
        '''runs a forward propagation of the biGRU.
        Returns a prediction of 1 or 0.'''
        x_embed = self.embedding(x).permute(1,0,2)
        out, _ = self.rnn(x_embed)
        forward_GRU = out[-1, :, :self.hidden_size]
        reverse_GRU = out[0, :, self.hidden_size:]
        
        merged_GRU = self.drop1(torch.cat((forward_GRU, reverse_GRU), 1))
        hid = merged_GRU @ self.hidden_weights 
        hid.add_(self.hidden_bias)
        prediction = torch.sigmoid(hid)
          
        return prediction

## Training, Held-out Validation and Grid Search

***
Here we use held-out validation to train our model. To find the optimal learning rate, hidden layer size and weight decay we perform a grid search over 27 different parameter combinations to determine their optimal values against the validation set. 

Note Adam is the chosen optimisation method as it slightly outperformed the other methods when tested on the four cell lines in the paper. 

In [19]:
device = torch.device("cpu")  
drop_prob = 0.5 # could also be trained in below grid search
max_auc = 0

# for loops for the parameter grid search
for l_rate in [0.02, 0.01, 0.005]:
    for hidden_size in [50, 80, 100]:
        for w_decay in [1e-2, 1e-3, 1e-5]:
            print("Testing Learning Rate: ", l_rate)
            print("Testing Hidden Size: ", hidden_size)
            print("Testing Weight Decay: ", w_decay)
            
            # define the model and optimiser and train
            auc_list = []
            model = biGRU(hidden_size, drop_prob).to(device)
            optimiser = torch.optim.Adam([model.hidden_weights, model.hidden_bias]+[param for param in model.parameters()], lr = l_rate, weight_decay = w_decay)
            model.train()
            for iter in range(30): # model training iterations
                for i, (data, label) in enumerate(train_loader): # iterate over batches
                    data = data.to(device)
                    label = label.to(device)
                    out = model(data)
                    loss = func.binary_cross_entropy(out, label)
                    optimiser.zero_grad()
                    loss.backward()
                    optimiser.step()

                   
                with torch.no_grad():
                    model.eval()
                    
                    # evaluate the model on the training set
                    auc = [] 
                    for i, (data, label) in enumerate(train_loader):
                        data = data.to(device)
                        label = label.to(device)
                        out = model(data)
                        pred = out.cpu().detach().numpy().reshape(out.shape[0])
                        labels = label.cpu().numpy().reshape(out.shape[0])
                        auc.append(metrics.roc_auc_score(labels, pred))

                    print("AUC training score: ", np.mean(auc))

                    # evaluate the model on the validation set
                    auc = []
                    for i, (data, label) in enumerate(valid_loader):
                        data = data.to(device)
                        label = label.to(device)
                        out = model(data)
                        pred = out.cpu().detach().numpy().reshape(out.shape[0])
                        labels = label.cpu().numpy().reshape(out.shape[0])
                        auc.append(metrics.roc_auc_score(labels, pred))

                    print("AUC validation score: ", np.mean(auc))
                    auc_list.append(np.mean(auc))
        
        # update the set of parameters if auc improves
        if max_auc < np.mean(auc_list):
            max_auc = np.mean(auc_list)
            opt_l_rate = l_rate
            opt_hidden_size = hidden_size
            opt_w_decay = w_decay
            print("Current optimal parameters: ", opt_hidden_size, opt_l_rate, opt_w_decay)            

print("Highest mean validation AUC: ", max_auc)   
print("Optimised parameters: ", opt_hidden_size, opt_l_rate, opt_w_decay)

Testing Learning Rate:  0.02
Testing Hidden Size:  50
Testing Weight Decay:  0.01
AUC training score:  0.8471776346938039
AUC validation score:  0.6672393386469929
AUC training score:  0.8981757949016173
AUC validation score:  0.7542944404189448
AUC training score:  0.8139798150256126
AUC validation score:  0.6881880376786773
AUC training score:  0.8835500038558941
AUC validation score:  0.7469523219814241
AUC training score:  0.9180484304575596
AUC validation score:  0.7829822211975496
AUC training score:  0.9365317760913549
AUC validation score:  0.7939612476121467
AUC training score:  0.9495727984663688
AUC validation score:  0.8024989197022594
AUC training score:  0.9008869166880565
AUC validation score:  0.751464383110467
AUC training score:  0.9571067087074985
AUC validation score:  0.8074366840129109
AUC training score:  0.9500025802613197
AUC validation score:  0.7943094196693234
AUC training score:  0.9580809800086375
AUC validation score:  0.798680739081747
AUC training score

## Retrain the BiGRU with Optimised Parameters

***
We now take the optimal parameters from the grid search (based upon highest AUC score) and use them to train our final model. 

In [148]:
device = torch.device("cpu")  

# optimised parameters from validation step
l_rate = opt_l_rate
w_decay = opt_w_decay
hidden_size = opt_hidden_size

# define the model and optimiser and train
model = biGRU(hidden_size, drop_prob).to(device)
optimiser = torch.optim.Adam([model.hidden_weights, model.hidden_bias]+[param for param in model.parameters()], lr = l_rate, weight_decay = w_decay)

for iter in range(30): # model training iterations
    model.train()
    for i ,(data, label) in enumerate(final_loader): # iterate over batches
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        loss = func.binary_cross_entropy(out, label)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    with torch.no_grad():
        model.eval()
        
        # evaluate the model on the training set
        auc = []
        for i, (data, label) in enumerate(final_loader):
            data = data.to(device)
            label = label.to(device)
            out = model(data)
            pred = out.cpu().detach().numpy().reshape(out.shape[0])
            labels = label.cpu().numpy().reshape(out.shape[0])
            auc.append(metrics.roc_auc_score(labels, pred))

        print("AUC training score: ", np.mean(auc))
     

AUC training score:  0.9460243537220197
AUC training score:  0.9754486039207744
AUC training score:  0.9856965822990307
AUC training score:  0.9954194189660874
AUC training score:  0.9973043748121169
AUC training score:  0.9986575267652295
AUC training score:  0.9994937040752285
AUC training score:  0.9993819249968321
AUC training score:  0.9994459019119492
AUC training score:  0.9996633379951203
AUC training score:  0.9995168883918933
AUC training score:  0.9998522115018825
AUC training score:  0.99998233792554
AUC training score:  0.9998166714266392
AUC training score:  0.9999139049690856
AUC training score:  0.9995609159328164
AUC training score:  0.9997169004953205
AUC training score:  0.9996131679872882
AUC training score:  0.9999410699868586
AUC training score:  0.9999792531735208
AUC training score:  0.9999911664582778
AUC training score:  0.9999696047509807
AUC training score:  0.999985293235206
AUC training score:  0.9994392244375149
AUC training score:  0.999914578612092
AUC 

## Evaluate the Optimised Model on the Test Set

***
Finally we will use the above trained model with optimised learning rate, weight decay and hidden layer size parameters to evaluate the performance of the trained model on an unseen test set.<br> Note there are many other metrics for testing a model's peformance; here we use the area under the ROC curve (AUC) which is used in the paper. 

In [142]:
with torch.no_grad():
    model.eval()
    auc = []
     
    for i, (data, label) in enumerate(test_loader):
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        pred = out.cpu().detach().numpy().reshape(out.shape[0])
        labels = label.cpu().numpy().reshape(out.shape[0])
        auc.append(metrics.roc_auc_score(labels, pred))
                          
    AUC_training=np.mean(auc)
    print("AUC test score: ", AUC_training)

AUC test score:  0.7831144696105587


### Final Notes

***

The influence of k-mer length, stride window size and embedding vector dimension size on model performance is still unknown. An improved intuition on how altering these parameters affects model performance as well as deepining an understanding of kmer vector embeddings are active areas of research and could provide a deeper insight into the inherent sequence structure and organisation of TF binding sites. <br>

Note: This model could work on *.fastq sequence data (e.g. from RNA-Seq experiments) enabling researchers to discover novel TF binding sites without any apriori knowledge about the transcription factors (the input files would have to be formatted similarly). 

Will Walters

The University of Melbourne